In [2]:
import os
%pwd

'c:\\Users\\y0vwts9\\OneDrive - Deere & Co\\Y0VWTS9\\Python\\Deep_Learning\\NLP\\PWSKILLS\\Text_Summarizer\\research'

In [3]:
os.chdir("../")
%pwd

'c:\\Users\\y0vwts9\\OneDrive - Deere & Co\\Y0VWTS9\\Python\\Deep_Learning\\NLP\\PWSKILLS\\Text_Summarizer'

In [4]:
## Setting up parameters in params.yaml


In [5]:
## Entity

from dataclasses import dataclass 
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    tokenizer_name: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    per_device_eval_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [6]:
## Configuration Manager

from src.TextSummarizer.constants import *
from src.TextSummarizer.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        # create_directories([self.config.artifacts_root])   #you can call .artifacts_root due to ConfigBox used in read.yaml fucntion
        create_directories([read_yaml(CONFIG_FILE_PATH).artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            tokenizer_name = config.tokenizer_name,
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            per_device_eval_batch_size = params.per_device_eval_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            evaluation_strategy = params.evaluation_strategy,
            eval_steps = params.eval_steps,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
        )
    
        return model_trainer_config


In [7]:
## Create components

import os
from TextSummarizer.logging import logger
from transformers import AutoTokenizer, DataCollatorForSeq2Seq, AutoModelForSeq2SeqLM, TrainingArguments, Trainer
from datasets import load_dataset, load_from_disk
import torch

class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)

    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

        # Load the data
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        # trainer_args = TrainingArguments(
        #                 output_dir=self.config.root_dir,
        #                 num_train_epochs=self.config.num_train_epochs,
        #                 warmup_steps=self.config.warmup_steps,
        #                 per_device_train_batch_size= self.config.per_device_train_batch_size,
        #                 per_device_eval_batch_size=self.config.per_device_eval_batch_size,
        #                 weight_decay=self.config.weight_decay,
        #                 logging_steps=self.config.logging_steps,
        #                 evaluation_strategy=self.config.evaluation_strategy,
        #                 eval_steps=self.config.eval_steps,
        #                 save_steps=self.config.save_steps,
        #                 gradient_accumulation_steps=self.config.gradient_accumulation_steps
        #             )
        
        trainer_args = TrainingArguments(
                        output_dir="pegasus-samsum",
                        num_train_epochs=1,
                        warmup_steps=500,
                        per_device_train_batch_size=1,
                        per_device_eval_batch_size=1,
                        weight_decay=0.01,
                        logging_steps=10,
                        evaluation_strategy="steps",
                        eval_steps=500,
                        save_steps=1e6,
                        gradient_accumulation_steps=16
                    )

        trainer = Trainer(
                    model = model_pegasus,
                    args = trainer_args,
                    tokenizer = tokenizer,
                    data_collator = seq2seq_data_collator,
                    train_dataset = dataset_samsum_pt["test"],    #taking test data for training as train daat size is more, will take time
                    eval_dataset = dataset_samsum_pt["validation"]
                )

        # Save the model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir, "pegasus-samsum-model"))

        # Save Tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir, "tokenizer"))


c:\Users\y0vwts9\AppData\Local\anaconda3\envs\TextS\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
## create Pipeline

try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config = model_trainer_config)
    model_trainer.train()   

except Exception as e:
    raise e

[2023-10-30 22:56:29,461: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-10-30 22:56:29,465: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-30 22:56:29,471: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-10-30 22:56:29,471: INFO: common: Created directory at: artifacts]
[2023-10-30 22:56:29,479: INFO: common: Created directory at: artifacts/model_trainer]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
